# Process preferences for Tutorials and CRAFT at FAT*2020

Author: [ChaTo](https://chato.cl/). Date: December 2019.

The allocation simulates a scenario in which people arrive in registration order and for each block, go to the event (tutorial or CRAFT) they prefer as long as that session has empty seats. Once they decide to attend an event, they stay for all the blocks used by that event.

* Individual preferences are sorted from highest to lowest, breaking ties randomly.
* People are processed in the order in which they registered for the conference, i.e., the field "*Original Response Date*" in the registration report generated by CEVENT.
   * When processing a person, blocks are processed in order. If a person is allowed in an event, all the blocks of that event are marked as occupied.
   * When processing a block:
      * If the person gave preferences, s/he is allocated to the event that has available seats for which s/he expressed the highest preference.
      * If the person did not gave preferences, s/he is allocated to the event with more capacity, we assume everybody can fit that event because in 2020 it's in a plenary room
      
This is a greedy procedure which may not generate an optimal solution. 

Please note that input files for on-site CRAFT sessions leave 5 seats unallocated to provide some flexibility.

**Attendees who expressed preferences** will have their assignment in output file ASSIGNMENT_PER_REGISTRANT. 

**Attendees who did not express preferences** will be listed in output file ASSIGNMENT_NO_PREFERENCE and may attend any tutorial or CRAFT that takes place in the large plenary room without further action. Otherwise, they may look for sessions with empty seats and attend those sessions.

In [5]:
import csv
import io
import datetime
import re
import random

In [8]:
# INPUT AND OUTPUT DIRECTORIES

INPUT_DIR = "data"
OUTPUT_DIR = "assignments"

#INPUT_DIR = "dummy_data"
#OUTPUT_DIR = "dummy_assignments"

In [9]:
# INPUT FILES AND PARAMETERS

REGISTRATIONS_FILE = INPUT_DIR + "/registrations.csv"
PREFERENCES_FILE = INPUT_DIR + "/preferences.csv"

TUTORIALS_FILE = INPUT_DIR + "/tutorials.csv"
TUTORIAL_BLOCKS = ["A", "B", "C"]

CRAFTS_FILE = INPUT_DIR + "/crafts.csv"
CRAFT_BLOCKS = ["D", "E", "F"]

PREF_ORDER = ["Highest", "High", "Indifferent", "Low", "Lowest"]
PREF_DEFAULT = "Indifferent"

In [10]:
# Output files

ASSIGNMENT_PER_REGISTRANT = OUTPUT_DIR + "/assignment_per_registrant.csv"
ASSIGNMENT_NO_PREFERENCE = OUTPUT_DIR + "/no_preference.csv"
ASSIGNMENT_PER_SESSION_PREFIX = OUTPUT_DIR + "/assignment_of_"
ASSIGNMENT_PER_SESSION_SUFFIX = ".csv"

# Read Tutorials and CRAFTs data

In [17]:
seen = {}
name2code = {}
code2name = {}

* *code*: a key for the event
* *name*: the name of the event
* *blocks*: a colon-separated list of blocks used by the event
* *capacity*: the maximum capacity in seats

In [18]:
tutorial_capacity = {}
tutorial_blocks = {}
tutorial_names = set()
block2tutorials = {}
block2starting_tutorials = {}

with io.open(TUTORIALS_FILE) as file:
    reader = csv.DictReader(file, delimiter=";")
    for row in reader:
        code = "T_" + row["code"]
        name = row["name"]
        blocks = row["blocks"]
        capacity = int(row["capacity"])
        
        assert(code not in seen)
        assert(len(code) == 7)
        seen[code] = True
                                
        blocks = blocks.split(":")
        assert(len(blocks)>=1 and len(blocks) <=3)
        for block in blocks:
            assert(block in TUTORIAL_BLOCKS)
            if block not in block2tutorials:
                block2tutorials[block] = []
            block2tutorials[block].append(code)

        starting_block = blocks[0]
        if starting_block not in block2starting_tutorials:
            block2starting_tutorials[starting_block] = []
        block2starting_tutorials[starting_block].append(code)

        name2code[name] = code
        code2name[code] = name
        tutorial_names.add(name)
        tutorial_capacity[code] = capacity
        tutorial_blocks[code] = blocks

print("Tutorials read")
for code in tutorial_capacity.keys():
    print("%s capacity %d blocks %s" % (code, tutorial_capacity[code], ",".join(tutorial_blocks[code])))
    

Tutorials read
T_xaind capacity 600 blocks A
T_fairn capacity 150 blocks A
T_ai360 capacity 75 blocks A,B
T_libre capacity 50 blocks A,B
T_twocs capacity 35 blocks A
T_po101 capacity 600 blocks B
T_posit capacity 150 blocks B
T_frien capacity 35 blocks B
T_meani capacity 600 blocks C
T_leapf capacity 150 blocks C
T_inter capacity 50 blocks C
T_probi capacity 75 blocks C
T_gdprw capacity 50 blocks C
T_total capacity 35 blocks C


In [19]:
craft_capacity = {}
craft_blocks = {}
craft_names = set()
block2crafts = {}
block2starting_crafts = {}

with io.open(CRAFTS_FILE) as file:
    reader = csv.DictReader(file, delimiter=";")
    for row in reader:
        code = "C_" + row["code"]
        name = row["name"]
        blocks = row["blocks"]
        capacity = int(row["capacity"])
        
        assert(code not in seen)
        assert(len(code) == 7)
        seen[code] = True
            
        blocks = blocks.split(":")
        assert(len(blocks)>=1 and len(blocks) <=3)
        for block in blocks:
            assert(block in CRAFT_BLOCKS)
            if block not in block2crafts:
                block2crafts[block] = []
            block2crafts[block].append(code)
            
        starting_block = blocks[0]
        if starting_block not in block2starting_crafts:
            block2starting_crafts[starting_block] = []
        block2starting_crafts[starting_block].append(code)

        name2code[name] = code
        code2name[code] = name
        craft_names.add(name)
        craft_capacity[code] = capacity
        craft_blocks[code] = blocks

print("CRAFTS read")
for code in craft_capacity.keys():
    print("%s capacity %d blocks %s" % (code, craft_capacity[code], ",".join(craft_blocks[code])))
    

CRAFTS read
C_scale capacity 600 blocks D
C_bridg capacity 35 blocks D
C_losti capacity 25 blocks D,E
C_offet capacity 35 blocks D,E,F
C_offde capacity 25 blocks F
C_fromt capacity 55 blocks D,E
C_burnd capacity 20 blocks D,E
C_algor capacity 20 blocks D,E
C_commu capacity 45 blocks D,E
C_notto capacity 600 blocks E
C_manif capacity 35 blocks E
C_rumps capacity 600 blocks F
C_cente capacity 70 blocks F
C_siteu capacity 45 blocks F
C_infra capacity 20 blocks F
C_hardw capacity 20 blocks F
C_offzi capacity 75 blocks F


# Read Registration and Preferences Data

## Read registration data

* *Confirmation #*: the registration code used as user key
* *Original Response Date*: their priority for taking a seat
* *Admission Item*: indicates if they signed up for tutorials, the conference (incl. CRAFTs) or both

In [27]:
registrants = []
reg2registrant = {}
reg_has_tutorials = {}
reg_has_crafts = {}

with io.open(REGISTRATIONS_FILE, "r", encoding='utf-8-sig') as file:
    reader = csv.DictReader(file, delimiter=",")
    for row in reader:
        
        assert("First Name" in row)
        assert("Last Name" in row)
        
        regnum = row["Confirmation #"]
        #assert(len(regnum) == 11)
        reg2registrant[regnum] = row
        
        regdate = row["Original Response Date"]
        assert(len(regdate)>0)
        date = datetime.datetime.strptime(regdate, "%m/%d/%Y %I:%M:%S %p")
        row["_priority"] = date
        
        regitems = row["Admission Item"]
        assert(len(regitems)>0)
        if bool(re.search("tutorials \+ conference", regitems)):
            reg_has_tutorials[regnum] = True
            reg_has_crafts[regnum] = True
        elif bool(re.search("tutorials only", regitems)):
            reg_has_tutorials[regnum] = True
            reg_has_crafts[regnum] = False
        elif bool(re.search("conference only", regitems)):
            reg_has_tutorials[regnum] = False
            reg_has_crafts[regnum] = True
        else:
            print("Unexpected value of 'Admission Item': %s" % regitems)
            assert(False)
            
        registrants.append(row)

        
print("Processed %d registrants" % len(registrants))

has_tutorials_count = sum([1 for regnum in reg_has_tutorials.keys() if reg_has_tutorials[regnum]])
has_crafts_count = sum([1 for regnum in reg_has_crafts.keys() if reg_has_crafts[regnum]])

print("Has tutorials: %d, has CRAFTs: %d" % (has_tutorials_count, has_crafts_count))

Processed 574 registrants
Has tutorials: 500, has CRAFTs: 569


## Read preferences data

* *Registration code*: the registration code used as user key
* *Tutorial preferences*: their interest in each tutorial (Highest to Lowest, with blank=Indifferent)
* *CRAFT preferences*: their interest in each CRAFT (Highest to Lowest, with blank=Indifferent)

In [33]:
code2preferences = {}
code2ordered_tutorial_preferences = {}
code2ordered_craft_preferences = {}

with io.open(PREFERENCES_FILE) as file:
    reader = csv.DictReader(file, delimiter=",")
    for row in reader:

        # Remove trailing spaces of regcode
        row["Registration code"] = row["Registration code"].strip()
        code = row["Registration code"]
        
        
        tutorial_preferences = {}
        craft_preferences = {}
        
        for key in row:
            if key == "Timestamp":
                assert(len(row[key]) > 0)
            elif key == "Email Address":
                assert(len(row[key]) > 0)
            elif key == "Registration code":
                if row[key] not in reg2registrant:
                    print("[Warning] Can't find registrant with code %s and e-mail %s" % (row[key], row["Email Address"]))
                    break
            elif key.startswith("Tutorial preferences") and reg_has_tutorials[code]:
                name_match = re.search("(Tutorial preferences) \[(.*)\]", key)
                tutorial_name = name_match.group(2)
                assert(tutorial_name in tutorial_names)
                assert(tutorial_name in name2code)
                tutorial_preferences[name2code[tutorial_name]] = row[key]
            elif key.startswith("Tutorial preferences") and not reg_has_tutorials[code]:
                if len(row[key])>0:
                    print("[Safe] Ignoring tutorial preference for non-tutorial user %s" % (code))

            elif key.startswith("CRAFT preferences") and reg_has_crafts[code]:
                name_match = re.search("(CRAFT preferences) \[(.*)\]", key)
                craft_name = name_match.group(2)
                assert(craft_name in craft_names)
                assert(craft_name in name2code)
                craft_preferences[name2code[craft_name]] = row[key]

            elif key.startswith("CRAFT preference") and not reg_has_crafts[code]:
                if len(row[key])>0:
                    print("[Safe] Ignoring CRAFT preference for non-CRAFT user %s" % (code))

                
            else:
                print("Unexpected column %s in %s" % (key, PREFERENCES_FILE))
                assert(False)
                
        # Complete tutorial preferences with Indifferent
        if len(tutorial_preferences.keys()) > 0:
            for key in tutorial_preferences.keys():
                if len(tutorial_preferences[key]) == 0:
                    tutorial_preferences[key] = PREF_DEFAULT

        # Complete CRAFT preferences with Indifferent
        if len(craft_preferences.keys()) > 0:
            for key in craft_preferences.keys():
                if len(craft_preferences[key]) == 0:
                    craft_preferences[key] = PREF_DEFAULT
                    
        # Order preferred tutorials from most to least pref., breaking ties arbitrarily (shuffle)
        ordered_tutorial_preferences = []
        tutorial_preferences_shuffled = list(tutorial_preferences.keys())
        random.shuffle(tutorial_preferences_shuffled)
        for level in PREF_ORDER:
            for pref_tut in tutorial_preferences_shuffled:
                if tutorial_preferences[pref_tut] == level:
                    ordered_tutorial_preferences.append(pref_tut)
                    
        # Order preferred CRAFTs from most to least pref., breaking ties arbitrarily (shuffle)
        ordered_craft_preferences = []
        craft_preferences_shuffled = list(craft_preferences.keys())
        random.shuffle(craft_preferences_shuffled)
        for level in PREF_ORDER:
            for pref_cra in craft_preferences_shuffled:
                if craft_preferences[pref_cra] == level:
                    ordered_craft_preferences.append(pref_cra)
        
        # Store preferences
        if len(ordered_tutorial_preferences) > 0:
            code2ordered_tutorial_preferences[code] = ordered_tutorial_preferences       
        
        if len(ordered_craft_preferences) > 0:
            code2ordered_craft_preferences[code] = ordered_craft_preferences    
            
        if len(ordered_tutorial_preferences) + len(ordered_craft_preferences) > 0:
            code2preferences[code] = row     
            
    
print("Read preferences for %d people" % len(code2preferences.keys()))    
print("Ordered tutorial preferences for %d people" % len(code2ordered_tutorial_preferences.keys()))    
print("Ordered CRAFT preferences for %d people" % len(code2ordered_craft_preferences.keys()))    


[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring tutorial preference for non-tutorial user NZNXL2BJX76
[Safe] Ignoring CRAF

# Process registrations for sign-up

In [34]:
def get_regdate(registrant):
    return registrant["_priority"]

registrants_with_preferences = []
registrants_without_preferences = []

for registrant in sorted(registrants, key=lambda x: get_regdate(x)):
    if registrant["Confirmation #"] in code2preferences:
        registrants_with_preferences.append(registrant)
    else:
        registrants_without_preferences.append(registrant)
        
print("Registrants with preferences    : %d" % len(registrants_with_preferences))
print("Registrants without preferences : %d" % len(registrants_without_preferences))

Registrants with preferences    : 199
Registrants without preferences : 375


In [35]:
idx = 0
for registrant in registrants_with_preferences:
    idx+=1
    print("%4d %s %s" % (idx, registrant["Confirmation #"], registrant["_priority"]))

   1 PLNSM97PX38 2019-10-11 19:33:00
   2 DLNFWD5T4Y5 2019-10-12 00:32:00
   3 GKNR65N3DNN 2019-10-13 18:54:00
   4 GDNZ4Q8QLZQ 2019-10-15 03:12:00
   5 XPNW8JS7JLX 2019-10-15 09:36:00
   6 H4NXM8WNSPB 2019-10-15 11:09:00
   7 NWN45XR4KQD 2019-10-15 18:05:00
   8 HNNDQHPC8KT 2019-10-16 09:24:00
   9 ZPNSX6PN6FY 2019-10-17 15:17:00
  10 VRNQVL4QV9P 2019-10-18 08:19:00
  11 N9NR2RWN6S7 2019-10-19 07:26:00
  12 H8NPZYCP93B 2019-10-19 21:41:00
  13 PDNRZCNHJ4C 2019-10-20 06:52:00
  14 JCNVHD6TG25 2019-10-20 06:58:00
  15 DPNYCLV3PMF 2019-10-20 08:35:00
  16 QYN79QS5KGB 2019-10-20 09:28:00
  17 KVN5CD7LZL6 2019-10-22 13:12:00
  18 LRN6G27GVN3 2019-10-22 19:23:00
  19 J6NPCF4DJ3X 2019-10-23 05:13:00
  20 N9NXSWFGC9T 2019-10-23 10:51:00
  21 NNNJM7WD7PW 2019-10-25 15:57:00
  22 GSNZPQ4WHLN 2019-10-29 13:21:00
  23 V5N2D3V56RX 2019-10-29 13:29:00
  24 VZNRV9NBQX3 2019-10-29 19:02:00
  25 L3NDXK8ZYGC 2019-10-30 00:41:00
  26 H5NY2J2MMSR 2019-10-30 12:51:00
  27 FBNS5L2JNNC 2019-10-30 15:54:00
 

In [36]:
# Process one by one

registrant2tutorials = {}
registrant2crafts = {}

tutorial_usage = {}
craft_usage= {}

for code in tutorial_capacity.keys():
    tutorial_usage[code] = 0

for code in craft_capacity.keys():
    craft_usage[code] = 0

# Now process all registrants
for registrant in registrants_with_preferences:
    code = registrant["Confirmation #"]
    tutorial_assignment = {}
    craft_assignment = {}
    
    registrant_is_busy = {}
    
    for block in TUTORIAL_BLOCKS:
        registrant_is_busy[block] = False
    for block in CRAFT_BLOCKS:
        registrant_is_busy[block] = False
          
    print("Processing %s Tutorials:%s CRAFTS:%s" % (code, reg_has_tutorials[code], reg_has_crafts[code]) )
    
    if code in code2preferences:
        # People who express preferences
        
        if code in code2ordered_tutorial_preferences and reg_has_tutorials[code]:
            # Tutorial
            ordered_tutorial_preferences = code2ordered_tutorial_preferences[code]
            print(" Tutorial preferences: %s" % ordered_tutorial_preferences)
            for block in TUTORIAL_BLOCKS:
                print("  Block %s" % block)
                if not registrant_is_busy[block]:
                    print("   Registrant is not busy during that block")
                    selected_tutorial = ""
                    for tutorial_code in ordered_tutorial_preferences:
                        if tutorial_code in block2starting_tutorials[block]:
                            if tutorial_capacity[tutorial_code] - tutorial_usage[tutorial_code] > 0:
                                selected_tutorial = tutorial_code
                                break
                    if len(selected_tutorial) > 0:
                        tutorial_assignment[block] = selected_tutorial
                        print("   Got assigned tutorial %s" % selected_tutorial)
                        for used_block in tutorial_blocks[selected_tutorial]:
                            print("    That marks as occupied block %s" % used_block)
                            registrant_is_busy[used_block] = True
                    else:
                        print("   No tutorial could be assigned")
          
        # CRAFT
        if code in code2ordered_craft_preferences and reg_has_crafts[code]:
            ordered_craft_preferences = code2ordered_craft_preferences[code]
            print(" CRAFT preferences: %s" % ordered_craft_preferences)
            for block in CRAFT_BLOCKS:
                print("  Block %s" % block)
                if not registrant_is_busy[block]:
                    print("   Registrant is not busy during that block")
                    selected_craft = ""
                    for craft_code in ordered_craft_preferences:
                        if craft_code in block2starting_crafts[block]:
                            if craft_capacity[craft_code] - craft_usage[craft_code] > 0:
                                selected_craft = craft_code
                                break
                    if len(selected_craft) > 0:
                        craft_assignment[block] = selected_craft
                        print("   Got assigned CRAFT %s" % selected_craft)
                        for used_block in craft_blocks[selected_craft]:
                            print("    That marks as occupied block %s" % used_block)
                            registrant_is_busy[used_block] = True
                    else:
                        print("   No CRAFT could be assigned")
        
        
    else:
        assert(False)
    
    # Assign and deduct from capacities    
    registrant2tutorials[code] = tutorial_assignment
    for block in tutorial_assignment:
        tutorial_usage[tutorial_assignment[block]] += 1
        
    registrant2crafts[code] = craft_assignment
    for block in craft_assignment:
        craft_usage[craft_assignment[block]] += 1

Processing PLNSM97PX38 Tutorials:True CRAFTS:True
 Tutorial preferences: ['T_inter', 'T_meani', 'T_fairn', 'T_posit', 'T_leapf', 'T_po101', 'T_frien', 'T_xaind', 'T_gdprw', 'T_twocs', 'T_total', 'T_probi', 'T_libre', 'T_ai360']
  Block A
   Registrant is not busy during that block
   Got assigned tutorial T_fairn
    That marks as occupied block A
  Block B
   Registrant is not busy during that block
   Got assigned tutorial T_posit
    That marks as occupied block B
  Block C
   Registrant is not busy during that block
   Got assigned tutorial T_inter
    That marks as occupied block C
 CRAFT preferences: ['C_algor', 'C_bridg', 'C_notto', 'C_commu', 'C_offet', 'C_manif', 'C_rumps', 'C_fromt', 'C_cente', 'C_scale', 'C_burnd', 'C_infra', 'C_hardw', 'C_offde', 'C_losti', 'C_offzi', 'C_siteu']
  Block D
   Registrant is not busy during that block
   Got assigned CRAFT C_algor
    That marks as occupied block D
    That marks as occupied block E
  Block E
  Block F
   Registrant is not bus

# Save assignments

In [37]:
session2writer = {}
sessionfiles = []

for tutorial_code in tutorial_capacity.keys():
    file = io.open(ASSIGNMENT_PER_SESSION_PREFIX + tutorial_code + ASSIGNMENT_PER_SESSION_SUFFIX, "w")    
    writer = csv.writer(file, delimiter=",")
    session2writer[tutorial_code] = writer
    sessionfiles.append(file)

for craft_code in craft_capacity.keys():
    file = io.open(ASSIGNMENT_PER_SESSION_PREFIX + craft_code + ASSIGNMENT_PER_SESSION_SUFFIX, "w")    
    writer = csv.writer(file, delimiter=",")
    session2writer[craft_code] = writer
    sessionfiles.append(file)
    
for writer in session2writer.values():
    writer.writerow(["First Name", "Last Name", "Email Address"])
    
# Registrants are sorted by priority
with io.open(ASSIGNMENT_PER_REGISTRANT, "w") as outfile:
    writer = csv.writer(outfile, delimiter=";")
    writer.writerow(["First Name", "Last Name", "Email Address", "Confirmation #", "Preferences", "Priority", "Tutorials", "CRAFTs"])
    
    for registrant in registrants_with_preferences:
        outrow = []
        
        regcode = registrant["Confirmation #"]
        regfn = registrant["First Name"]
        regln = registrant["Last Name"]
        regemail = registrant["Email Address"]
        
        outrow.append(regfn)
        outrow.append(regln)
        outrow.append(regemail)
        outrow.append(regcode)
        outrow.append("Yes" if regcode in code2preferences else "No")
        outrow.append(registrant["_priority"])
        
        # Process assignments
        
        tutorial_assignment = registrant2tutorials[regcode]
        craft_assignment = registrant2crafts[regcode]
        print("%s: %s %s" % (regcode, tutorial_assignment, craft_assignment))
        
        # List of tutorials in order of blocks
        tutorial_assigned_names = []
        for tut_block in TUTORIAL_BLOCKS:
            if tut_block in tutorial_assignment:
                tutorial_assigned_names.append(code2name[tutorial_assignment[tut_block]])
                session2writer[tutorial_assignment[tut_block]].writerow([regfn, regln, regemail])                
        outrow.append(" (AND) ".join(tutorial_assigned_names))
        
        # List of CRAFTs in order of blocks
        craft_assigned_names = []
        for cra_block in CRAFT_BLOCKS:
            if cra_block in craft_assignment:
                craft_assigned_names.append(code2name[craft_assignment[cra_block]]) 
                session2writer[craft_assignment[cra_block]].writerow([regfn, regln, regemail])
        outrow.append(" (AND) ".join(craft_assigned_names))
        
        writer.writerow(outrow)

# Registrants are sorted by priority
with io.open(ASSIGNMENT_NO_PREFERENCE, "w") as outfile:
    writer = csv.writer(outfile, delimiter=";")
    writer.writerow(["First Name", "Last Name", "Email Address"])
    
    for registrant in registrants_without_preferences:
        outrow = [registrant["First Name"], registrant["Last Name"], registrant["Email Address"]]
        writer.writerow(outrow)

for sessionfile in sessionfiles:
    sessionfile.close()

PLNSM97PX38: {'A': 'T_fairn', 'B': 'T_posit', 'C': 'T_inter'} {'D': 'C_algor', 'F': 'C_rumps'}
DLNFWD5T4Y5: {'A': 'T_twocs', 'B': 'T_frien', 'C': 'T_inter'} {'D': 'C_algor', 'F': 'C_siteu'}
GKNR65N3DNN: {'A': 'T_xaind', 'B': 'T_posit', 'C': 'T_meani'} {'D': 'C_scale', 'E': 'C_notto', 'F': 'C_cente'}
GDNZ4Q8QLZQ: {'A': 'T_twocs', 'B': 'T_frien', 'C': 'T_meani'} {'D': 'C_losti', 'F': 'C_offde'}
XPNW8JS7JLX: {'A': 'T_twocs', 'B': 'T_posit', 'C': 'T_meani'} {'D': 'C_algor', 'F': 'C_offzi'}
H4NXM8WNSPB: {'A': 'T_ai360', 'C': 'T_probi'} {'D': 'C_offet'}
NWN45XR4KQD: {'A': 'T_twocs', 'B': 'T_po101', 'C': 'T_leapf'} {'D': 'C_algor', 'F': 'C_siteu'}
HNNDQHPC8KT: {'A': 'T_libre', 'C': 'T_meani'} {'D': 'C_fromt', 'F': 'C_offde'}
ZPNSX6PN6FY: {'A': 'T_libre', 'C': 'T_meani'} {'D': 'C_commu', 'F': 'C_cente'}
VRNQVL4QV9P: {} {'D': 'C_fromt', 'F': 'C_offde'}
N9NR2RWN6S7: {'A': 'T_twocs', 'B': 'T_po101', 'C': 'T_gdprw'} {'D': 'C_offet'}
H8NPZYCP93B: {} {'D': 'C_scale', 'E': 'C_notto', 'F': 'C_rumps'}
